In [36]:
import os
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
truth = {}
with open('/usr0/home/ptejaswi/TextVQA_0.5.1_val.json', 'rb') as fp:
    for x in json.loads(fp.read())['data']:
        truth[x['question_id']] = x

print("Truths:", len(truth))

Truths: 5000


In [43]:
qids = """37988
37122
38733
36133
37833
35131
36752
36798
35699
39320
35584
37243
39336
39317
37723
38719
37458
37459
35217
35354
34992
36450
37487
38766
38956
36570
34891
37142
38927
35103
35494
36137
39486
36009
37268
37103
35607
38166
34613
37844
39596
37096
37862
38764
36133
36715
38841
36321
38212
37523
35167
37452
37733
34980
38865
38267
35063
38688
35088
36600
34892
35716
36134
37146
35016
37743
36483
36448
35058
34826
38742
36827
36792
38236
36076
38346
35150
37773
39068
35564
38155
36080
37491
39287
38753
34797
36695
36758
35701
37695
37116
39197
37832
37163
36714
34649
37236
37570
35597
35568
34663
37323""".split('\n')

In [44]:
len(qids)

102

In [58]:
towrite = []

for q in qids:
    q = int(q)
    d = truth[q]
    towrite.append({
        'question_id': q,
        'question': d['question'],
        'img_url': d['flickr_original_url'],
        'answers': d['answers']
    })
    
with open('./selftalk/samples.json', 'w') as fp:
    fp.write(json.dumps(towrite))
    
print("Written.")

Written.


In [59]:
len(towrite)

102

In [13]:
valdata = np.load('/usr1/home/ptejaswi/TAP/data/imdb/m4c_textvqa/imdb_val_ocr_en.npy', 
                  allow_pickle=True, encoding='latin1')
qidmap = {x['question_id']: x for x in valdata[1:]}

In [30]:
valdata[1]

{'question': 'what is the brand of this camera?',
 'image_id': '003a8ae2ef43b901',
 'image_classes': ['Cassette deck',
  'Printer',
  'Medical equipment',
  'Computer mouse',
  'Scale',
  'Telephone',
  'Camera',
  'Ipod',
  'Remote control'],
 'flickr_original_url': 'https://farm2.staticflickr.com/4/5566811_bc00d504a6_o.jpg',
 'flickr_300k_url': 'https://farm2.staticflickr.com/4/5566811_bc00d504a6_o.jpg',
 'image_width': 1024,
 'image_height': 664,
 'answers': ['nous les gosses',
  'dakota',
  'clos culombu',
  'dakota digital',
  'dakota',
  'dakota',
  'dakota digital',
  'dakota digital',
  'dakota',
  'dakota'],
 'question_tokens': ['what', 'is', 'the', 'brand', 'of', 'this', 'camera'],
 'question_id': 34602,
 'set_name': 'val',
 'image_name': '003a8ae2ef43b901',
 'image_path': 'train/003a8ae2ef43b901.jpg',
 'feature_path': '003a8ae2ef43b901.npy',
 'valid_answers': ['nous les gosses',
  'dakota',
  'clos culombu',
  'dakota digital',
  'dakota',
  'dakota',
  'dakota digital',
  '

In [60]:
# Annotations from Aman.
with open('./selftalk/samples_annotated.json') as fp:
    annotated = json.loads(fp.read())

In [61]:
len(annotated)

102

In [22]:
from copy import deepcopy

custom_manual = [valdata[0]]
for d in annotated:
    qid = d['question_id']
    meta = deepcopy(qidmap[qid])
    meta['question'] = meta['question'] + ' ' + d['annotation']
    custom_manual.append(meta)
    
custom_manual = np.array(custom_manual)

In [24]:
np.save('./selftalk/custom_manual.npy', custom_manual, allow_pickle=True)

In [25]:
len(custom_manual)

103

# Results

In [72]:
results = []
annot = {}
with open('./save/m4c_base_custom/m4c_textvqa_m4c_split/reports/m4c_textvqa_run_val_2022-03-31T22:44:12.json') as fp:
    for r in json.loads(fp.read()):
        results.append({
            'pred_answer': r['answer'],
            'gt_answers': qidmap[r['question_id']]['valid_answers']
        })
        annot[r['question_id']] = r

In [73]:
results[0]

{'pred_answer': 'steven jobs',
 'gt_answers': ['mensch, john',
  'john',
  'jo mensch',
  'unanswerable',
  'steve jobs',
  'steve jobs',
  'jobs',
  'answering does not require reading text in the image',
  'jobs ',
  'steve jobs']}

In [74]:
len(annot)

101

In [87]:
noann = {}
with open('./save/m4c_base_val/m4c_textvqa_m4c_split/reports/m4c_textvqa_run_val_2022-03-13T20:55:49.json') as fp:
    for r in json.loads(fp.read()):
        noann[r['question_id']] = r

In [113]:
noclue = 0
for qid, a in annot.items():
    base = noann[qid]
    meta = qidmap[qid]
    
    if base['answer'] != a['answer']:
        print(qid)
        print("question:", meta['question'])
        print("base:", base['answer'], base['pred_source'])
        print("annotation:", [x for x in annotated if x['question_id'] == qid][0]['annotation'])
        print("anno:", a['answer'], a['pred_source'])
        print("url:", meta['flickr_300k_url'])
        print("ans:", meta['answers'])
        noclue += 'clue' in [x for x in annotated if x['question_id'] == qid][0]['annotation']
        print()
        
print("No clues:", noclue)

39317
question: what is the title of the red burgundy book?
base: the invisible man ['VOCAB', 'OCR', 'VOCAB']
annotation: The title of the red burgundy book is on the side.
anno: the treasure of looking man ['VOCAB', 'OCR', 'VOCAB', 'OCR', 'OCR']
url: https://c1.staticflickr.com/5/4020/4708026558_f67ce8364f_z.jpg
ans: ['invisible man', 'pygmalion', 'little women', 'pygmalion', 'pygmalion', 'pygmailon', 'candida', 'candida', 'pygmalion', 'partillion']

36137
question: what time is it on the watch?
base: 10:07 ['VOCAB']
annotation: Time of the watch can be read with position of the two hands.
anno: 1:53 ['VOCAB']
url: https://c6.staticflickr.com/9/8169/7892744740_53ba62e802_z.jpg
ans: ['10:10', '10:10', '10:10', '10:10', '10:10', '10:10', '10:10', '10:10', '10:10', '10:10']

37452
question: what is the book with the gray cover ?
base: neither poverty nor poverty nor possessions ['OCR', 'OCR', 'OCR', 'OCR', 'OCR', 'OCR']
annotation: Title of a book is on the top of the book
anno: neither 

In [110]:
a

{'question_id': 39317,
 'image_id': '9d74f00ea0548b56',
 'answer': 'the treasure of looking man',
 'pred_source': ['VOCAB', 'OCR', 'VOCAB', 'OCR', 'OCR']}

In [106]:
print("base performance:", evaluator.eval_pred_list([{
    'pred_answer': a['answer'],
    'gt_answers': qidmap[qid]['answers']
} for qid, a in noann.items() if qid in annot], full=True))

base performance: [0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.3, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.3, 0.0, 0.3, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [108]:
print("annot performance:", evaluator.eval_pred_list([{
    'pred_answer': a['answer'],
    'gt_answers': qidmap[qid]['answers']
} for qid, a in annot.items() if qid in annot]))

annot performance: 0.103960396039604


In [88]:
evaluator.eval_pred_list?

In [90]:
base_answers = evaluator.eval_pred_list([{
    'pred_answer': a['answer'],
    'gt_answers': qidmap[qid]['answers']
} for qid, a in noann.items()], full=True)

In [99]:
combined = zip(base_answers, noann.keys())

In [100]:
corrects = [t[1] for t in combined if t[0] == 1]

In [101]:
len(corrects)

2180

In [102]:
rand30 = np.random.choice(corrects, size=30, replace=False)

In [103]:
rand30

array([36533, 37387, 38630, 39187, 38004, 39361, 36013, 37126, 35960,
       35144, 37333, 38898, 35482, 38636, 37076, 38744, 37148, 35475,
       39409, 39226, 38869, 38512, 39545, 39382, 37673, 38662, 38209,
       36272, 37822, 35544])